# VoxelGrid

In this notebook we are going to learn about how to construct and visualize a grid of voxels from a point cloud.

# Imports

In [1]:
import numpy as np

In [2]:
from pyntcloud import PyntCloud

# Setup

We are going to **load** a 3D model of an ankylosaurus from the provided `examples/data.`

You could load other point cloud in any of the supported [point cloud formats](http://pyntcloud.readthedocs.io/en/latest/io.html).

You can learn more about reading and writing point clouds in the `examples/[io]` notebooks.

In [3]:
anky = PyntCloud.from_file("data/ankylosaurus_mesh.ply")
anky

PyntCloud
34820 points with 6 scalar fields
69636 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: 0.029972486197948456, 1.1250594854354858, 2.887650489807129
Other attributes:

From the description, we can see that this is a 3D mesh with 69636 faces. 

We can visualize the mesh as follows:

In [ ]:
anky.plot(mesh=True, backend="threejs")

![mesh](data/images/structures-voxelgrid-1.png)

And convert it to a point cloud by sampling 200.000 random points from the surface.

You cand learn more about hoy to convert a triangular mesh into a point cloud in the `examples/[sampling]` notebooks.

In [4]:
anky_cloud = anky.get_sample(
    "mesh_random", 
    n=200000, 
    rgb=True, 
    normals=True, 
    as_PyntCloud=True)

We can visualize the new point cloud:

In [ ]:
anky_cloud.plot()

![anky_cloud](data/images/structures-voxelgrid-2.png)

# VoxelGrid creation

A `VoxelGrid` is a structure generated by subdividing the minimum bounding box of a point cloud into **voxels**.

![voxelgrid](data/images/structures-voxelgrid-3.png)

You can create a `VoxelGrid` from a `PyntCloud` using the `add_structure` with the `voxelgrid` identifier and some other optional arguments.

By default, a 1x1x1 `VoxelGrid` will be created, which is not very usefull.

`pyntcloud` supports different methods for specifying how a `VoxelGrid` shold be created.

You can indicate the number of voxels per axis that you want, the desired size per axis of each voxel or a combination of both.

There are some examples bellow:

### Regular VoxelGrid using number of voxels per axis

In [5]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", n_x=64, n_y=64, n_z=64)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![64x64x64](data/images/structures-voxelgrid-4.png)

### Irregular VoxelGrid using number of voxels per axis

In [6]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", n_x=8, n_y=16, n_z=32)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![8x16x32](data/images/structures-voxelgrid-5.png)

### Regular VoxelGrid using sizes

In [7]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", size_x=0.15, size_y=0.15, size_z=0.15)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![0.15x0.15x0.15](data/images/structures-voxelgrid-6.png)

### Irregular VoxelGrid using sizes

In [8]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", size_x=0.01, size_y=0.05, size_z=0.15)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![0.01x0.05x0.15](data/images/structures-voxelgrid-7.png)

### VoxelGrid mixing number of voxels and sizes

Note that, for the same axis, **sizes** override the value of the number of voxels.

In [9]:
voxelgrid_id = anky_cloud.add_structure(
    "voxelgrid",
    n_x=32, 
    size_y=0.05, 
    # n_z gets overrided by size_z and it'll be ignored
    n_z=2, 
    size_z=0.01)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![32x0.05x0.01](data/images/structures-voxelgrid-8.png)

### VoxelGrid with irregular bounding box

By default, the bounding box of the voxelgrid will be adapted to make all sides of equal length. This option can be disabled as follows:

In [10]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", n_x=64, n_y=64, n_z=64, regular_bounding_box=False)

voxelgrid = anky_cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.plot(d=3, mode="density", cmap="hsv")

![irregular bounding box](data/images/structures-voxelgrid-9.png)

Note that the now the shape of the voxels is not equal for all axis, in constrast with the same construction without the regular_bounding_box argument.

In [11]:
voxelgrid.shape

[0.015778325498104095, 0.0053504230454564095, 0.0094350166618824]

# VoxelGrid Information

Once you have constructed a `VoxelGrid`, you can access many information about the structure.

Bellow is the code to get the actual VoxelGrid instance once you have added it to a PyntCloud.

In [12]:
voxelgrid_id = anky_cloud.add_structure("voxelgrid", n_x=64, n_y=64, n_z=64)

voxelgrid = anky_cloud.structures[voxelgrid_id]

### Corresponding voxel per axis for each point in the point cloud

In [13]:
voxelgrid.voxel_x

array([32, 15, 29, ..., 47, 37, 25], dtype=int64)

In [14]:
voxelgrid.voxel_y

array([39, 36, 23, ..., 27, 23, 36], dtype=int64)

In [15]:
voxelgrid.voxel_z

array([17, 14, 30, ..., 36, 17, 21], dtype=int64)

### Unique voxel number for each point in the point cloud

In [16]:
voxelgrid.voxel_n

array([133585,  63758, 120286, ..., 194276, 153041, 104725], dtype=int64)

### Number of voxels per axis

This might be usefull in case you use size arguments for constructing the structure

In [17]:
voxelgrid.x_y_z

[64, 64, 64]

### Total number of voxels

In [18]:
voxelgrid.n_voxels

262144

### Shape of the voxel

In [19]:
voxelgrid.shape

[0.015778325498104095, 0.015778325498104095, 0.015778325498104095]

### Minimum and maximum coordinates of each voxel per axis

In [20]:
min_max_x, min_max_y, min_max_z = voxelgrid.segments

In [21]:
min_max_x

array([-0.43055117, -0.41477285, -0.39899452, -0.38321619, -0.36743787,
       -0.35165954, -0.33588122, -0.32010289, -0.30432457, -0.28854624,
       -0.27276792, -0.25698959, -0.24121127, -0.22543294, -0.20965461,
       -0.19387629, -0.17809796, -0.16231964, -0.14654131, -0.13076299,
       -0.11498466, -0.09920634, -0.08342801, -0.06764968, -0.05187136,
       -0.03609303, -0.02031471, -0.00453638,  0.01124194,  0.02702027,
        0.04279859,  0.05857692,  0.07435524,  0.09013357,  0.1059119 ,
        0.12169022,  0.13746855,  0.15324687,  0.1690252 ,  0.18480352,
        0.20058185,  0.21636017,  0.2321385 ,  0.24791683,  0.26369515,
        0.27947348,  0.2952518 ,  0.31103013,  0.32680845,  0.34258678,
        0.3583651 ,  0.37414343,  0.38992175,  0.40570008,  0.42147841,
        0.43725673,  0.45303506,  0.46881338,  0.48459171,  0.50037003,
        0.51614836,  0.53192668,  0.54770501,  0.56348334,  0.57926166])

### Voxel Centers

In [22]:
voxelgrid.voxel_centers

array([[-0.42266202,  0.622388  ,  2.4117832 ],
       [-0.42266202,  0.622388  ,  2.4275615 ],
       [-0.42266202,  0.622388  ,  2.4433398 ],
       ...,
       [ 0.5713725 ,  1.6164225 ,  3.374261  ],
       [ 0.5713725 ,  1.6164225 ,  3.3900392 ],
       [ 0.5713725 ,  1.6164225 ,  3.4058175 ]], dtype=float32)

# VoxelGrid Methods

### VoxelGrid.query

You can `query` the voxelgrid with an array of points in order to find out in which number of voxel those points
would lie.

In [23]:
some_points = np.array(
    [[0.5, 0.5, 0.5],
     [0.1, 0.1, 2.3],
     [0.49, 0.49, 0.49]])

In [24]:
voxelgrid.query(some_points)

array([237568, 135168, 237568], dtype=int64)

### VoxelGrid.get_feature_vector

You can get the 3D array representing the VoxelGrid using different modes.

#### binary

The `binary` mode will return a 3D array with 0s on empty voxels and 1s on voxels that have one point or more.

In [25]:
binary_feature_vector = voxelgrid.get_feature_vector(mode="binary")

In [26]:
binary_feature_vector.shape

(64, 64, 64)

You can count the number of occupied voxels as follow:

In [27]:
binary_feature_vector.sum()

4413.0

Which should match the number of unique voxel identifiers:

In [28]:
len(np.unique(voxelgrid.voxel_n))

4413

#### density

The `density` mode will return a 3D array with a value normalized between 0 and 1 representing the percentage of points that each voxel contains.

In [29]:
density_feature_vector = voxelgrid.get_feature_vector(mode="density")

In [30]:
density_feature_vector.shape

(64, 64, 64)

The density feature vector should sum to 1.

In [31]:
density_feature_vector.sum()

0.9999999999999999

#### x_max, y_max, z_max

The `x_max`, `y_max`, `z_max` modes will return a 3D array indicating the maximum coordinate value inside a voxel, for the corresponding axis.

In [32]:
x_max_feature_vector = voxelgrid.get_feature_vector(mode="x_max")

In [33]:
x_max_feature_vector.shape

(64, 64, 64)

The maximum and minimum `x` value of all voxels should match the maximum and minimum `x` value of the point cloud.

In [35]:
x_max_feature_vector.max(), max(anky_cloud.points["x"])

(0.5792616605758667, 0.5792616605758667)

#### x_mean, y_mean, z_mean

The `x_mean`, `y_mean`, `z_mean` modes will return a 3D array indicating the mean coordinate value inside a voxel, for the corresponding axis.

In [36]:
z_mean_feature_vector = voxelgrid.get_feature_vector(mode="z_mean")

c:\users\daavoo\desktop\pyntcloud\pyntcloud\structures\voxelgrid.py:184: RuntimeWarning: invalid value encountered in true_divide
  vector = np.nan_to_num(voxel_sum / voxel_count)


In [37]:
z_mean_feature_vector.shape

(64, 64, 64)

The maximum and minimum `z` value of all voxels should be bellow/above the maximum and minimum `z` values of the point cloud.

In [38]:
z_mean_feature_vector.max(), max(anky_cloud.points["z"])

(3.20985860824585, 3.210721015930176)